In [ ]:
import os
import re
import time
import random
import math
import requests
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [ ]:
# 读取需要的环境变量
load_dotenv()
# link info
# 样品检测信息
ZMJL_PAGE = os.getenv('ZMJL_PAGE')
ZMJL_PAGE_TAIL = os.getenv('ZMJL_PAGE_TAIL')

# 表层样品
BC_PCH_DSH_URL = os.getenv('BC_PCH_DSH_URL')
BC_PCH_YSH_URL = os.getenv('BC_PCH_YSH_URL')

BC_PCH_DSH_PAGE_URL = os.getenv('BC_PCH_DSH_PAGE_URL')
BC_PCH_YSH_PAGE_URL = os.getenv('BC_PCH_YSH_PAGE_URL')
BC_PCH_PAGE_URL_TAIL = os.getenv('BC_PCH_PAGE_URL_TAIL')

# 剖面样品

PM_PCH_DSH_URL = os.getenv('PM_PCH_DSH_URL')
PM_PCH_YSH_URL = os.getenv('PM_PCH_YSH_URL')

PM_PCH_DSH_PAGE_URL = os.getenv('PM_PCH_DSH_PAGE_URL')
PM_PCH_YSH_PAGE_URL = os.getenv('PM_PCH_YSH_PAGE_URL')
PM_PCH_PAGE_URL_TAIL = os.getenv('PM_PCH_PAGE_URL_TAIL')

# 水团样品
ST_PCH_DSH_URL = os.getenv('ST_PCH_DSH_URL')
ST_PCH_YSH_URL = os.getenv('ST_PCH_YSH_URL')

ST_PCH_DSH_PAGE_URL = os.getenv('ST_PCH_DSH_PAGE_URL')
ST_PCH_YSH_PAGE_URL = os.getenv('ST_PCH_YSH_PAGE_URL')
ST_PCH_DSH_PAGE_URL_TAIL = os.getenv('ST_PCH_DSH_PAGE_URL_TAIL')

# 检测结果
JCJG_WLZB_PAGE = os.getenv('JCJG_WLZB_PAGE') 
JCJG_HXZB_PAGE = os.getenv('JCJG_HXZB_PAGE') 
JCJG_PAGE_MID = os.getenv('JCJG_PAGE_MID')
JCJG_PAGE_TAIL = os.getenv('JCJG_PAGE_TAIL')
# headers info 
headers = {
    "Accept": os.getenv("ACCEPT"),
    "Accept-Encoding": os.getenv("ACCEPT_ENCODING"),
    "Accept-Language": os.getenv("ACCEPT_LANGUAGE"),
    "Authorization": os.getenv("AUTHORIZATION"),
    "Connection": os.getenv("CONNECTION"),
    "Cookie": os.getenv("COOKIE"),
    "Host": os.getenv("HOST"),
    "Referer": os.getenv("REFERER"),
    "Sec-Ch-Ua": os.getenv("SEC_CH_UA"),
    "Sec-Ch-Ua-Mobile": os.getenv("SEC_CH_UA_MOBILE"),
    "Sec-Ch-Ua-Platform": os.getenv("SEC_CH_UA_PLATFORM"),
    "Sec-Fetch-Dest": os.getenv("SEC_FETCH_DEST"),
    "Sec-Fetch-Mode": os.getenv("SEC_FETCH_MODE"),
    "Sec-Fetch-Site": os.getenv("SEC_FETCH_SITE"),
    "User-Agent": os.getenv("USER_AGENT")}


In [ ]:
# 获取当前页面cookie信息
def get_cookie():
    """获取当前页面cookie信息"""
    # 使用JavaScript获取Cookie
    cookie_script = """
    return document.cookie;
    """
    cookie_value = driver.execute_script(cookie_script)
    return cookie_value
# 更新cookie信息
def update_cookie(headers):
    """更新cookie信息"""
    # 获取当前页面cookie信息
    cookie_value = get_cookie()
    # 更新headers
    headers.update({"Cookie": cookie_value})
    return headers

# 获取转码记录总数量
def get_trans_num(headers):
    """获取当前转码记录总数量"""
    url = f'{ZMJL_PAGE}1{ZMJL_PAGE_TAIL}'
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']['total']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
    
# 获取转码信息
def get_trans_info(page_number,headers):
    """获取当前页面转码信息"""
    url = f'{ZMJL_PAGE}{page_number}{ZMJL_PAGE_TAIL}'
    # 发送GET请求
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']['records']
    else:
        print("请求失败，状态码：", response.status_code)
# 获取各状态审核数量
def get_status_num(type_url,headers):
    """获取各状态审核数量"""
    # 发送GET请求
    response = requests.get(type_url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']['count']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
# 获取指定状态指定页面的批次信息及批次编码
def get_status_pc_info(url,headers):
    """获取指定页面的批次编码"""
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        info_list = data['result']['records']
        return info_list
    else:
        print("请求失败，状态码：", response.status_code)
        return False    

# 获取指定批次号下的物理指标
def get_physical_info(url,headers):
    """获取指定批次号下的物理指标"""
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']['records'],data['result']['total']
    else:
        print("请求失败，状态码：", response.status_code)
        return False
    
# 获取指定批次号下的化学指标
def get_chemical_info(url,headers):
    """获取指定批次号下的化学指标"""
    response = requests.get(url, headers=headers)
    # 检查响应状态码
    if response.status_code == 200:
        # 解析响应内容
        data = response.json()
        return data['result']['records'],data['result']['total']
    else:
        print("请求失败，状态码：", response.status_code)
        return False

In [ ]:
# 配置浏览器
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Selenium\AutomationProfile"
# chrome.exe --remote-debugging-port=9999 --user-data-dir="D:\Program Files\ChromeDir"
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "localhost:9998")
driver = webdriver.Chrome(options=options)

driver.implicitly_wait(60)

In [ ]:
# 保存路径
save_path = r"F:\collection_spb_info\ZK"
# 如果没有该文件夹，则创建
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
# 表层样各状态审核数量
headers = update_cookie(headers)
bc_wait_number = get_status_num(BC_PCH_DSH_URL,headers)
bc_audi_number = get_status_num(BC_PCH_YSH_URL,headers)

In [ ]:
bc_wait_number,bc_audi_number

In [ ]:
# 更新headers
headers = update_cookie(headers)
bc_wait_info_list = []
bc_wait_pcnum_list = []
# 获取表层待审核批次信息
for one_loop in tqdm(range(math.ceil(bc_wait_number/50))):
    temp_bc_wait_info = get_status_pc_info(f'{BC_PCH_DSH_PAGE_URL}{one_loop+1}{BC_PCH_PAGE_URL_TAIL}',headers)
    temp_bc_wait_pc_num = [_['pch'] for _ in temp_bc_wait_info]
    bc_wait_info_list.extend(temp_bc_wait_info)
    bc_wait_pcnum_list.extend(temp_bc_wait_pc_num)

In [ ]:
# 更新headers
headers = update_cookie(headers)
bc_audit_info_list = []
bc_audit_pcnum_list = []
# 获取表层已审核批次信息
for one_loop in tqdm(range(math.ceil(bc_audi_number/50))):
    temp_bc_audit_info = get_status_pc_info(f'{BC_PCH_YSH_PAGE_URL}{one_loop+1}{BC_PCH_PAGE_URL_TAIL}',headers)
    temp_bc_audit_pc_num = [_['pch'] for _ in temp_bc_audit_info]
    bc_audit_info_list.extend(temp_bc_audit_info)
    bc_audit_pcnum_list.extend(temp_bc_audit_pc_num)

In [ ]:
# 保存待审核批次号
with open(os.path.join(save_path,f'bc_dsh_{len(bc_wait_pcnum_list)}.json'), 'w') as f:
    json.dump(bc_wait_pcnum_list, f, indent=4)
# 保存已审核批次号
with open(os.path.join(save_path,f'bc_ysh_{len(bc_audit_pcnum_list)}.json'), 'w') as f:
    json.dump(bc_audit_pcnum_list, f, indent=4)

In [ ]:
# 表层所有批次信息
bc_all_info_list = bc_audit_info_list+bc_wait_info_list
df_bc_info = pd.DataFrame(bc_all_info_list)
df_bc_info.head(1)

In [ ]:
# 保存表层待审核数据
df_bc_info.to_excel(os.path.join(save_path, 'bc_info.xlsx'),index=False)

In [ ]:
# 获取表层物理指标
bc_physical_info_list = []
bc_all_pcnum_list = bc_audit_pcnum_list+bc_wait_pcnum_list
# 更新headers
headers = update_cookie(headers)
for one_loop in tqdm(bc_all_pcnum_list):
    url = f'{JCJG_WLZB_PAGE}{JCJG_PAGE_MID}{one_loop}{JCJG_PAGE_TAIL}'
    # 获取指标信息
    bc_physical_info = get_physical_info(url, headers)
    # 检测是否包含完整个批次数据
    if bc_physical_info[1] <= 50:
        bc_physical_info_list.extend(bc_physical_info[0])
    else:
        print(f'{one_loop}批次数据获取得到{bc_physical_info[1]}条，请检查!')
        break

In [ ]:
df_bc_physical_info =pd.DataFrame(bc_physical_info_list)
df_bc_physical_info.head(1)

In [ ]:
# 保存数据
df_bc_physical_info.to_excel(os.path.join(save_path, 'bc_physical_info.xlsx'),index=False)

In [ ]:
# 获取表层化学指标
bc_chemical_info_list = []
# 更新headers
headers = update_cookie(headers)
for one_loop in tqdm(bc_all_pcnum_list):
    url = f'{JCJG_HXZB_PAGE}{JCJG_PAGE_MID}{one_loop}{JCJG_PAGE_TAIL}'
    # 获取指标信息
    bc_chemical_info = get_chemical_info(url, headers)
    # 检测是否包含完整个批次数据
    if bc_chemical_info[1] <= 50:
        bc_chemical_info_list.extend(bc_chemical_info[0])
    else:
        print(f'{one_loop}批次数据获取得到{bc_chemical_info[1]}条，请检查!')
        break

In [ ]:
df_bc_chemical_info =pd.DataFrame(bc_chemical_info_list)
df_bc_chemical_info.head(1)

In [ ]:
df_bc_chemical_info.to_excel(os.path.join(save_path, "df_bc_chemical_info.xlsx"),index=False)

In [ ]:
# 获取转码信息
# 获取总的转码数量
total_trans_num = get_trans_num(headers)
# 计算需要循环的次数
loop_tran_num = math.ceil(total_trans_num / 50)


In [ ]:
# 测试信息
trans_info_test = get_trans_info(1,headers)
trans_info_test[0]

In [ ]:

# 循环获取转码信息
trans_info_list = []
# 更新headers
headers = update_cookie(headers)
for one_loop in tqdm(range(0, loop_tran_num)):
    # 获取转码信息
    temp_trans_info = get_trans_info(one_loop+1, headers)
    # 添加到列表
    trans_info_list.extend(temp_trans_info)
    # 随机休眠
    # time.sleep(random.random())

In [ ]:
df_trans_info = pd.DataFrame(trans_info_list)
df_trans_info.head(1)

In [ ]:
# 保存转码信息
df_trans_info.to_excel(os.path.join(save_path, 'trans_info.xlsx'),index=False)

# 检查heraders并更新

In [ ]:

headers = update_cookie(headers)

In [ ]:
headers

In [ ]:
# 获取当前账户下的待审核和已审核编码，并写入文件
# 更新headers
headers = update_cookie(headers)
audit_point_list = get_page_number(get_audit_num(headers),headers)
wait_point_list = get_wait_page_number(get_wait_num(headers),headers)
all_point = audit_point_list + wait_point_list
# 提取编码
loop_all_point = [list(item.keys())[0] for item in all_point]

In [ ]:
json_save_path = os.path.join(save_path,f'poind_id_{len(all_point)}.json')
# 写入JSON文件
with open(json_save_path, 'w') as json_file:
    json.dump(all_point, json_file, indent=4)
# 读取原始JSON文件
with open(json_save_path, 'r') as json_file:
    data_dict = json.load(json_file)

# 初始化字典，用于存储值为'1'的数据
data_with_1 = {}
# 遍历列表，每个元素都是一个字典
for item_dict in data_dict:
    for key, value in item_dict.items():
        if value == '0':
            pass
        elif value == '1':
            data_with_1[key] = value
# 存储值为剖面样的数据到JSON文件
with open(os.path.join(save_path,f'data_with_1_{len(data_with_1)}.json'), 'w') as json_file:
    json.dump(data_with_1, json_file, indent=4)


In [ ]:
len(all_point),len(wait_point_list),len(audit_point_list)

In [ ]:
# 测试一个点位的基本信息
base_info = get_base_info(loop_all_point[0],headers)
base_info

In [ ]:
# 获取基本信息
# 更新headers
headers = update_cookie(headers)
base_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_base_info = get_base_info(one_point,headers)['result']
    # 随机休眠
    time.sleep(random.random())
    base_info_list.append(one_point_base_info)


In [ ]:
df_base_info = pd.DataFrame(base_info_list)
df_base_info.head(1)

In [ ]:
# 保存基本信息
df_base_info.to_excel(os.path.join(save_path,'base_info.xlsx'),index=False)

In [ ]:
# 测试一个点位的图片信息
img_info = get_img_info(loop_all_point[0],img_headers)
img_info

In [ ]:
# 获取图片链接信息
# 更新headers
img_headers = update_cookie(img_headers)
img_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_img_info = get_img_info(one_point,img_headers)['result']
    # 为链接增加域名
    prefix = BASE_NAME
    updated_list = [{**item, 'wjlj': f"{prefix}{item['wjlj']}"} for item in one_point_img_info]
    # 随机休眠
    time.sleep(random.random())
    img_info_list+=updated_list

In [ ]:
df_img_info = pd.DataFrame(img_info_list)
df_img_info.head(1)

In [ ]:
# 保存媒体信息
df_img_info.to_excel(os.path.join(save_path, 'img_info.xlsx'), index=False)

In [ ]:
# 测试一个点位的立地调查信息
ldtj_info = get_ldtj_info(loop_all_point[0],headers)
ldtj_info

In [ ]:
# 获取立地条件信息
# 更新headers
headers = update_cookie(headers)
ldtj_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_ldtj_info = get_ldtj_info(one_point,headers)['result']
    ldtj_info_list.append(one_point_ldtj_info)
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_ldtj_info = pd.DataFrame(ldtj_info_list)
df_ldtj_info.head(1)

In [ ]:
# 保存立地条件信息
df_ldtj_info.to_excel(os.path.join(save_path,'ldtj_info.xlsx'),index=False)

In [ ]:
# 测试一个点位的采土袋信息
ctd_info = get_ctd_info(loop_all_point[0],headers)
ctd_info

In [ ]:
# 获取采土袋信息
# 更新headers
headers = update_cookie(headers)
ctd_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_ctd_info = get_ctd_info(one_point,headers)['result']
    ctd_info_list+=one_point_ctd_info
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_ctd_info = pd.DataFrame(ctd_info_list)
df_ctd_info.head(1)

In [ ]:
# 保存采土袋信息
df_ctd_info.to_excel(os.path.join(save_path, 'ctd_info.xlsx'), index=False)

In [ ]:
# 测试一个点位的施肥信息
sf_info = get_sf_info(loop_all_point[0],headers)
sf_info

In [ ]:
# 获取施肥信息
# 更新headers
headers = update_cookie(headers)
sf_info_list = []
for one_point in tqdm(loop_all_point):
    one_point_sf_info = get_sf_info(one_point,headers)['result']
    sf_info_list+=one_point_sf_info
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_sf_info = pd.DataFrame(sf_info_list)
df_sf_info.head(1)

In [ ]:
# 保存施肥信息
df_sf_info.to_excel(os.path.join(save_path, 'sf_info.xlsx'), index=False)

# 剖面信息

In [ ]:
# 剖面点位信息
# 读取原始JSON文件
with open(os.path.join(save_path,f'data_with_1_{len(data_with_1)}.json'), 'r') as json_file:
    pm_dict = json.load(json_file)
pm_point_id = [_ for _,__ in pm_dict.items()]


In [ ]:
# 测试剖面点位的信息
pm_info = get_pm_info(pm_point_id[0],headers)
pm_info

In [ ]:
# 获取剖面信息
# 更新headers
headers = update_cookie(headers)
pm_info_list = []
for one_point in tqdm(pm_point_id):
    one_point_pm_info = get_pm_info(one_point,headers)['result']
    pm_info_list.append(one_point_pm_info)
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_pm_info = pd.DataFrame(pm_info_list)
df_pm_info.head(1)

In [ ]:
# 保存剖面信息
df_pm_info.to_excel(os.path.join(save_path, 'pm_info.xlsx'), index=False)

In [ ]:
# 测试剖面点位发生层信息
pm_fc_info = get_pmfc_info(pm_point_id[0],headers)
pm_fc_info

In [ ]:
# 获取剖面发生层信息
# 更新headers
headers = update_cookie(headers)
pm_fc_info_list = []
for one_point in tqdm(pm_point_id):
    one_point_pm_fc_info = get_pmfc_info(one_point,headers)['result']
    pm_fc_info_list+=one_point_pm_fc_info
    # 随机休眠
    time.sleep(random.random())

In [ ]:
df_pm_fc_info = pd.DataFrame(pm_fc_info_list)
df_pm_fc_info.head(1)

In [ ]:
# 保存剖面发生层信息
df_pm_fc_info.to_excel(os.path.join(save_path, 'pm_fc_info.xlsx'), index=False)

# TODO